In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re


In [23]:
url = "https://www.netflix-codes.com/fr"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
linkcode = [] #luu tat ca code theo tung the loai phim
for temp in soup.find_all('ul', class_='pl-4 list-inside list-disc space-y-1'):
    for link in temp.find_all("a"):
        linkcode.append(link['href'])
print(linkcode)


['https://www.netflix.com/browse/genre/46576', 'https://www.netflix.com/browse/genre/10702', 'https://www.netflix.com/browse/genre/2125', 'https://www.netflix.com/browse/genre/11828', 'https://www.netflix.com/browse/genre/1568', 'https://www.netflix.com/browse/genre/7442', 'https://www.netflix.com/browse/genre/43040', 'https://www.netflix.com/browse/genre/77232', 'https://www.netflix.com/browse/genre/10118', 'https://www.netflix.com/browse/genre/20541', 'https://www.netflix.com/browse/genre/8985', 'https://www.netflix.com/browse/genre/9584', 'https://www.netflix.com/browse/genre/43048', 'https://www.netflix.com/browse/genre/7700', 'https://www.netflix.com/browse/genre/11881', 'https://www.netflix.com/browse/genre/2729', 'https://www.netflix.com/browse/genre/9302', 'https://www.netflix.com/browse/genre/2653', 'https://www.netflix.com/browse/genre/10695', 'https://www.netflix.com/browse/genre/452', 'https://www.netflix.com/browse/genre/11146', 'https://www.netflix.com/browse/genre/3063',

In [77]:
link_item = [] #luu link dan toi moi bo phim
for link in linkcode:
    link_url = link
    
    try:
        link_response = requests.get(link_url)
        link_response.raise_for_status()  # Kiểm tra trạng thái phản hồi, nếu có lỗi sẽ ném ngoại lệ

        link_soup = BeautifulSoup(link_response.content, 'html.parser')
        if link_soup.find('div', class_='error-page not-found'):
            continue
        
        if link_soup.find_all('li', class_='nm-collections-row-gallery-li'):
            for temp in link_soup.find_all('li', class_='nm-collections-row-gallery-li'):
                for temp2 in temp.find_all("a"):
                    href = temp2['href']
                    if href not in link_item: #kiem tra link da ton tai hay chua
                        link_item.append(href)
        else:
            for temp in link_soup.find_all('li', class_='nm-content-horizontal-row-item'):
                for temp2 in temp.find_all("a"):
                    href = temp2['href']
                    if href not in link_item:
                        link_item.append(href)
                        
    
    except requests.exceptions.RequestException:
        # Nếu không thể truy cập liên kết, bỏ qua và tiếp tục vòng lặp
        continue


In [112]:
len(link_item)

4468

In [113]:
def time_to_minutes(time_str):
    # Tách thời gian thành giờ và phút
    parts = time_str.split()
    
    total_minutes = 0
    
    for part in parts:
        if 'h' in part:
            # Lấy giờ và chuyển thành phút
            hours = int(part.replace('h', ''))
            total_minutes += hours * 60
        elif 'm' in part:
            # Lấy phút
            minutes = int(part.replace('m', ''))
            total_minutes += minutes
    
    return total_minutes



In [118]:

ds = []
for link in link_item:
    link_url = link
    link_response = requests.get(link_url)
    link_soup = BeautifulSoup(link_response.content, 'html.parser')
    id = re.findall(r'\d+',link)
    try:
        name =  link_soup.find('h1', class_='title-title').text
    except AttributeError:
        continue
    try: 
        year = link_soup.find('span', class_='title-info-metadata-item item-year').text
    except AttributeError:
        continue
    try:    
        rating = link_soup.find('span', class_='maturity-number').text
    except AttributeError:
        continue
    try:
        time = link_soup.find('span', class_='duration').text
    except AttributeError:
        continue
    try:
        describle = link_soup.find('div', class_='title-info-synopsis').text
    except  AttributeError:
        continue
    try:
        starring = link_soup.find('span', class_='title-data-info-item-list').text
    except AttributeError:
        continue
    creator =[]
    genres = []
    country = []
    country_mapping = {
            "Vietnamese": "Vietnam",
            "Egyptian": "Egypt",
            "Taiwanese":"China",
            "Norwegian": "Norway",
            "Singaporean": "Singapore",
            "Tamil-Language":"Singapore",
            "Chinese": "China",
            "Hong Kong": "China",
            "US": "United States",
            "Spanish": "Spain",
            "French": "France",
            "Filipino": "Philippines",
            "Hindi-Language": "India",
            "Kuwaiti": "Kuwait",
            "Saudi": "Saudi Arabia",
            "Polish": "Poland",
            "Political": "Poland",
            "African": "South Africa",
            "South African": "South Africa",
            "Argentinian": "Argentina",
            "Austrian": "Austria",
            "Belgian": "Belgium",
            "Bengali-Language": "Belgium",
            "Brazilian": "Brazil",
            "Canadian": "Canada",
            "Chilean": "Chile",
            "Colombian": "Colombia",
            "Thai": "Thailand",
            "Korean": "Korea",
            "Indian": "India",
            "Telugu-Language": "India",
            "Japanese": "Japan",
            "English": "United Kingdom",
            "Czech": "Czech Republic",
            "Irish": "Ireland",
            "Malaysian": "Malaysia",
            "Italian": "Italy",
            "British": "United Kingdom",
            "Swedish": "Sweden",
            "German": "Germany",
            "Dutch": "Netherlands",
            "Australian": "Australia",
            "Indonesian": "Indonesia",
            "Mexican": "Mexico",
            "Portuguese": "Portugal",
            "Syrian":"Syria",
            "Lebanese": "Lebanon",
            "Malayalam-Language Movies": "India",
            "Turkish": "Turkey",
            "Russian": "Russia",
            "Danish": "Denmark",
            "Peruvian": "Peru",
            "Nollywood": "Nigeria",

        }
     
    for gen in link_soup.find_all('span', class_='more-details-item item-genres'):
        # print(gen.text)
        # count = 0
        country_temp = []
        for temp, country_name in country_mapping.items():
            temp = str(temp)

         # Sử dụng re.findall để tìm tất cả các sự xuất hiện của temp trong genres
            
            if re.findall(rf'\b{temp}\b', gen.text) :
                    # count +=1
                    if country_name not in country_temp:
                        country_temp.append(country_name)
        if len(country_temp)>0:

            country.append(country_temp)

        else:
            genres.append(gen.text.replace(",",""))
        # print(country)
        # print("\n")
    # country = '' .join(country)   
    genres = ',' .join(genres)    
    if link_soup.find('div', class_='title-data-info-item item-creators'):
        creator = link_soup.find('div', class_='title-data-info-item item-creators').find('span', class_='title-data-info-item-list').text
    type 
    if re.findall(r'\bTV\b',genres):
        type = 'TV Show'
    else:
        type = 'Movie'
        if time_to_minutes(time) == 0:
            type = 'TV Show'
        else: 
            time = str(time_to_minutes(time)) + ' min'
    
        
    data = {
            'id':id[0],
            'type':type,
            'name':name,
            'creator': None if len(creator) == 0 else creator,
            'starring':starring,
            'year':year,
            'rating':rating,
            'time':time,
            'genres': genres,
            'country':None if len(country) == 0 else country[0][0],
            'describle':describle,

        }
    ds.append(data)

    

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [115]:
len(ds)


3784

In [93]:

min_error_index = link_item.index('https://www.netflix.com/vn-en/title/80990612')
min_error_index


679

In [116]:
df=pd.DataFrame(ds)
df.to_csv("netflix.csv",encoding='utf-8',index=False)

In [ ]:
for link in link_item[0:3]:
    # link_url = link
    # link_response = requests.get(link_url)
    # link_soup = BeautifulSoup(link_response.content, 'html.parser')
    print(link)

https://www.netflix.com/vn-en/title/60001220
https://www.netflix.com/vn-en/title/1056189
https://www.netflix.com/vn-en/title/686619


In [ ]:
country_mapping = {
            "Viet": "Vietnam",
            "Chine": "China",
            "US": "United States",
            "Kore": "Korea",
            "Indi":"India",
            "Jana":"Japan",
            "Eng": "United Kingdom",
            "Czech":"Czech Republic",
            "Irish":"Ireland",
            "Malaysian":"Malaysia",
            "Italian":"Italy",
            "British":"United Kingdom",
            "Swedish":"Sweden",
            "German":"Germany",
            "Dutch":"Netherlands",
            "Australian":"Australia",

        }
for temp in country_mapping:
    print(country_mapping[temp])

Vietnam
China
United States
Korea
India
Japan
United Kingdom
Czech Republic
Ireland
Malaysia
Italy
United Kingdom
Sweden
Germany
Netherlands
Australia
